In [3]:
import sqlite3
import os

In [4]:
os.listdir()

['README.md',
 '.ipynb_checkpoints',
 'buddymove_holidayiq.csv',
 'rpg_db.sqlite3',
 'schema.png',
 'Untitled.ipynb']

In [6]:
conn = sqlite3.connect('rpg_db.sqlite3')

In [7]:
curs = conn.cursor()

In [14]:
character_count = 'SELECT COUNT(name) FROM charactercreator_character;'

In [19]:
#total characters
print(curs.execute(character_count).fetchall())

[(302,)]


In [17]:
print(curs.fetchall())

[(302,)]


In [20]:
conn.commit()
conn.close()

In [32]:
#total for each class
subclasses = ['cleric', 'fighter', 'mage', 'necromancer', 'thief']
conn = sqlite3.connect('rpg_db.sqlite3')
for subclass in subclasses:
    curs = conn.cursor()
    if subclass == 'necromancer':
        class_count = 'SELECT COUNT(mage_ptr_id) FROM charactercreator_'+subclass+';'
    else:
        class_count = 'SELECT COUNT(character_ptr_id) FROM charactercreator_'+subclass+';'
    print(f'{subclass}: ', curs.execute(class_count).fetchall()[0])
conn.commit()
conn.close()

cleric:  (75,)
fighter:  (68,)
mage:  (108,)
necromancer:  (11,)
thief:  (51,)


In [36]:
#total items
conn = sqlite3.connect('rpg_db.sqlite3')
curs = conn.cursor()
item_count = 'SELECT count(item_id) from armory_item'
print("total armory items", curs.execute(item_count).fetchall()[0])
conn.close()

total armory items (174,)


In [40]:
#total weapons
conn = sqlite3.connect('rpg_db.sqlite3')
curs = conn.cursor()
weapon_count = 'SELECT count(item_ptr_id) from armory_weapon'
print("total armory weapons", curs.execute(weapon_count).fetchall()[0])
conn.close()

total armory weapons (37,)


In [41]:
#items not weapons
174-37

137

In [45]:
#counting the number of items each character has (first 20 rows)
conn = sqlite3.connect('rpg_db.sqlite3')
curs = conn.cursor()
item_count = 'SELECT character_id, count(id) from \
              charactercreator_character_inventory GROUP by character_id LIMIT 20'
print("total items by character", curs.execute(item_count).fetchall())
conn.close()

total items by character [(1, 3), (2, 3), (3, 2), (4, 4), (5, 4), (6, 1), (7, 5), (8, 3), (9, 4), (10, 4), (11, 3), (12, 3), (13, 4), (14, 4), (15, 4), (16, 1), (17, 5), (18, 5), (19, 3), (20, 1)]


In [48]:
#counting the number of weapons each character has (first 20 rows)
conn = sqlite3.connect('rpg_db.sqlite3')
curs = conn.cursor()
item_count = 'SELECT character_id, count(item_ptr_id) from \
              charactercreator_character_inventory, armory_weapon, armory_item \
               WHERE armory_weapon.item_ptr_id = armory_item.item_id \
               GROUP by character_id LIMIT 20'
print("total weapons by character", curs.execute(item_count).fetchall())
conn.close()

total weapons by character [(1, 111), (2, 111), (3, 74), (4, 148), (5, 148), (6, 37), (7, 185), (8, 111), (9, 148), (10, 148), (11, 111), (12, 111), (13, 148), (14, 148), (15, 148), (16, 37), (17, 185), (18, 185), (19, 111), (20, 37)]


## Assignment - Part 1, Querying a Database
- How many total Characters are there?
- How many of each specific subclass?
- How many total Items?
- How many of the Items are weapons? How many are not?
- How many Items does each character have? (Return first 20 rows)
- How many Weapons does each character have? (Return first 20 rows)
- On average, how many Items does each Character have?
- On average, how many Weapons does each character have?

In [78]:
#On average, how many items each character has
#must use count(character_id) as ci then take avg(ci) 
#instead of avg(count(character_id)) even though it is allowed
#in other sql implementations
#https://database.guide/avg-calculate-the-average-value-of-a-column-in-mysql/
conn = sqlite3.connect('rpg_db.sqlite3')
curs = conn.cursor()
avg_items = 'SELECT avg(ci) FROM (\
            SELECT *, count(character_id) as ci\
            FROM armory_item, charactercreator_character_inventory \
            WHERE armory_item.item_id = charactercreator_character_inventory.item_id\
            GROUP by character_id)'
print("average items by character", curs.execute(avg_items).fetchall())
conn.close()

average items by character [(2.9735099337748343,)]


In [76]:
#On average, how many Weapons each character has
conn = sqlite3.connect('rpg_db.sqlite3')
curs = conn.cursor()
avg_weapons = 'SELECT avg(ci) FROM (\
            SELECT *, count(character_id) as ci\
            FROM armory_item, armory_weapon, charactercreator_character_inventory \
            WHERE armory_item.item_id = armory_weapon.item_ptr_id AND armory_item.item_id\
            = charactercreator_character_inventory.item_id\
            GROUP by character_id)'
print("average weapons by character", curs.execute(avg_weapons).fetchall())
conn.close()

average weapons by character [(1.3096774193548386,)]


## Assigment - Part 2, Making and populating a Database

Load the data (use `pandas`) from the provided file `buddymove_holidayiq.csv`
(the [BuddyMove Data
Set](https://archive.ics.uci.edu/ml/datasets/BuddyMove+Data+Set)) - you should
have 249 rows, 7 columns, and no missing values. The data reflects the number of
place reviews by given users across a variety of categories (sports, parks,
malls, etc.).

Using the standard `sqlite3` module:

- Open a connection to a new (blank) database file `buddymove_holidayiq.sqlite3`
- Use `df.to_sql`
  ([documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html))
  to insert the data into a new table `review` in the SQLite3 database

Then write the following queries (also with `sqlite3`) to test:

- Count how many rows you have - it should be 249!
- How many users who reviewed at least 100 `Nature` in the category also
  reviewed at least 100 in the `Shopping` category?
- (*Stretch*) What are the average number of reviews for each category?


In [49]:
import pandas as pd

In [50]:
df = pd.read_csv('buddymove_holidayiq.csv')

In [53]:
df.shape

(249, 7)

In [54]:
df.isnull().sum()

User Id      0
Sports       0
Religious    0
Nature       0
Theatre      0
Shopping     0
Picnic       0
dtype: int64

In [57]:
df.head()

,User Id,Sports,Religious,Nature,Theatre,Shopping,Picnic
0,User 1,2,77,79,69,68,95
1,User 2,2,62,76,76,69,68
2,User 3,2,50,97,87,50,75
3,User 4,2,68,77,95,76,61
4,User 5,2,98,54,59,95,86


In [59]:
conn = sqlite3.connect('buddymove_holidayiq.sqlite3')

In [60]:
df.to_sql('review', conn)

ValueError: Table 'review' already exists.

In [61]:
curs = conn.cursor()

In [65]:
row_count = 'SELECT COUNT([User Id]) FROM review;'

In [68]:
print("Total number of rows: ", curs.execute(row_count).fetchall())

Total number of rows:  [(249,)]


In [67]:
big_nature_shopping = 'SELECT COUNT([User Id]) FROM review WHERE (Nature >= 100) and (Shopping >= 100);'

In [70]:
print("At least 100 Nature and 100 Shopping reviews: ", curs.execute(big_nature_shopping).fetchall())

At least 100 Nature and 100 Shopping reviews:  [(78,)]


In [75]:
#average for each category
categories = ['Sports', 'Religious', 'Nature', 'Theatre', 'Shopping', 'Picnic']
for cat in categories:
    curs = conn.cursor()
    avg_cat = 'SELECT avg('+cat+') FROM review;'
    print(f'{cat}: ', curs.execute(avg_cat).fetchall()[0])
conn.commit()
conn.close()

Sports:  (11.987951807228916,)
Religious:  (109.77911646586345,)
Nature:  (124.51807228915662,)
Theatre:  (116.37751004016064,)
Shopping:  (112.63855421686748,)
Picnic:  (120.40160642570281,)
